# 如何向链的状态添加值

:::info 先决条件

本指南假设您熟悉以下概念：
- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)
- [链式可运行对象](/docs/how_to/sequence/)
- [并行调用可运行对象](/docs/how_to/parallel/)
- [自定义函数](/docs/how_to/functions/)
- [传递数据](/docs/how_to/passthrough)

:::

[传递数据](/docs/how_to/passthrough)通过链的步骤的另一种方法是保持链状态的当前值不变，同时在给定键下分配一个新值。[`RunnablePassthrough.assign()`](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html#langchain_core.runnables.passthrough.RunnablePassthrough.assign) 静态方法接收一个输入值，并添加传递给 assign 函数的额外参数。

这在常见的 [LangChain 表达式语言](/docs/concepts/lcel) 模式中很有用，即加性地创建一个字典以用作后续步骤的输入。

这是一个例子：

In [ ]:
%pip install --upgrade --quiet langchain langchain-openai

import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

In [2]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'extra': {'num': 1, 'mult': 3}, 'modified': 2}

让我们分解一下这里发生的事情。

- 链的输入是 `{"num": 1}`。这被传递给一个 `RunnableParallel`，它以该输入并行调用它接收到的 runnable。
- `extra` 键下的值被调用。`RunnablePassthrough.assign()` 保留了输入字典中的原始键 (`{"num": 1}`)，并分配了一个名为 `mult` 的新键。其值为 `lambda x: x["num"] * 3)`，即 `3`。因此，结果是 `{"num": 1, "mult": 3}`。
- `{"num": 1, "mult": 3}` 被返回给 `RunnableParallel` 调用，并被设为 `extra` 键的值。
- 同时，`modified` 键被调用。结果是 `2`，因为 lambda 函数从其输入中提取一个名为 `"num"` 的键并加一。

因此，结果是 `{'extra': {'num': 1, 'mult': 3}, 'modified': 2}`。

## 流式处理

此方法的一个便捷之处在于，它允许值在可用时立即通过。为了展示这一点，我们将使用 `RunnablePassthrough.assign()` 在检索链中立即返回源文档：

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

generation_chain = prompt | model | StrOutputParser()

retrieval_chain = {
    "context": retriever,
    "question": RunnablePassthrough(),
} | RunnablePassthrough.assign(output=generation_chain)

stream = retrieval_chain.stream("where did harrison work?")

for chunk in stream:
    print(chunk)

{'question': 'where did harrison work?'}
{'context': [Document(page_content='harrison worked at kensho')]}
{'output': ''}
{'output': 'H'}
{'output': 'arrison'}
{'output': ' worked'}
{'output': ' at'}
{'output': ' Kens'}
{'output': 'ho'}
{'output': '.'}
{'output': ''}


我们可以看到第一个块包含原始的 `"question"`，因为它是即时可用的。第二个块包含 `"context"`，因为检索器是第二个完成的。最后，`generation_chain` 的输出会随着它的可用而以块的形式流式传输。

## 下一步

现在您已经学会了如何将数据传递通过链来帮助格式化在链中流动的数据。

要了解更多信息，请参阅本节中关于 runnables 的其他操作指南。